# Pandas

Pandas is a python library providing rich functionality on top of numpy. In addition to 'Excel like' tables, Pandas works well with numpy constructs and scikit-learn.

For more information, the docs are available at:
http://pandas.pydata.org/pandas-docs/stable/

In [ ]:
import pandas as pd
import numpy as np

# Pandas Series

Pandas series are like 1-dimensional numpy arrays, except that they are _labeled_, or have indices. In addition, the elements can be numeric, bools, strings, date time objects, functional objects, etc.

In [ ]:
#Can create the Pandas series from a Python list similar to Numpy
flt_series = pd.Series( [1.0,-2, .43434] )
flt_series

In [ ]:
#Can also pass in a Numpy ndarray
flt_series = pd.Series( np.random.random(5) )
flt_series

In [ ]:
# Dtype here is int64
int_series = pd.Series( np.random.random_integers(0,5,5) )
int_series

In [ ]:
# Dtype here is object
str_series = pd.Series([x*2 for x in 'abcd'])
str_series

In [ ]:
tup_series = pd.Series([(x,x+1) for x in range(4)])
tup_series

In [ ]:
fun_series = pd.Series( [map for x in range(5) ])
fun_series

In [ ]:
# Unlike numpy, all elements of the series do not have to be the same type
mix_series = pd.Series( [1.0, -2, map, 'aa', np.nan])
mix_series

In [ ]:
# Can use generic head() or head(# elements)
print mix_series.head()
print mix_series.head(2)

In [ ]:
# Can use generic tail() or tail(# elements)
print mix_series.tail()
print mix_series.tail(2)

In [ ]:
# Check for null/Nan on each element
mix_series.isnull()

In [ ]:
# Can mix in Numpy types/operations with pandas objects
np.any( mix_series.isnull() ) #Are ANY of the elements null

## Exercise

**1) Create two pd series with `a = [1,2,3,4]` and `b = [2.0, 3.0, 1.0,-1.2]`. Use `head(1)` to view only the first item of `a`**

**2) Create a new pd series c = [-2, 0, 3, np.nan ]. Add `a + c`. What do you get?**

Hint: you can use the `+` operator, or `np.add`

**3) What does comparing the 4th element of c to np.nan evaluate to? What does imply when comparing objects with multiple nan values? Test this theory using np.all**

**4) Create a pd Series where a = [1,2,3,'c',None,np.nan]. We have seen two methods so far for evaluating nans. `np.isnan` and `pd.isnull`. Evaluate both functions on `a`. What do you see?**

# Indexes


Each row of the pandas series has an index by default. In fact, you can specify your own indices 

Why have indices? Think of them as not only ways to conveniently label rows, but also can perform fast lookups, grouping operations, descriptive stats associated with these indices.

(These are not the same as the _labels_ mentioned in supervised learning)


Indices can be strings, integers, or even time series

In [ ]:
indSeries1 = pd.Series(np.random.random(5), 
                       index=['CA','AK','IL','IN','NY'])

In [ ]:
indSeries2 = pd.Series(np.random.random(3), 
                       index=['CA','IL','WA'])

In [ ]:
#Really cool! Matches the indices together and adds up by index
indSeries1 + indSeries2

In [ ]:
datSeries = pd.Series(np.random.random(5), 
                      index=pd.date_range('2015-01-01','2015-06-01',
                                           freq='m'))
datSeries

In [ ]:
# Just as a 'you can do this', if the index is a pd.date_range object,
# then you can resample by frequency
datSeries.resample('q') #Resample by quarter
datSeries.resample('d') #Resample by day, will NA fill
datSeries.resample('d', fill_method='ffill') #Resample by day, will forward will

Indexes need not be strings. They can be integers as well

In [ ]:
datSeries = pd.Series(np.random.random(10),
                     index=np.random.randint(0,1000,10))
datSeries

In [ ]:
#Pandas series have multiple aggregation/description methods:
# http://pandas.pydata.org/pandas-docs/version/0.17.1/api.html#computations-descriptive-stats


datSeries = pd.Series([1,2,5,3,5,3,2], 
                      index=[x for x in 'abcdefg'])
datSeries.value_counts()
datSeries.unique()
datSeries.nunique()
datSeries.all()
datSeries.any()

## Exercise

**1) Create two Pandas Series with various indices. `a` is a series with `arange(5)` having indices ['CA','IL','PA','IN','WA']. `b`  is a series with `arange(5)` having indices ['CA','MN','IL','IN','WA'].**

**Add `a + b`. What happens in the result?**

**2) Repeat the sum, but instead of `a+b`, try `a.add(b, fill_value=0)`. How does the result change?**

**3) Create two series. `a` is `arange(1,5)` having index `['a','b','c',a']`. `b` is `arange(1,5)` having index `['a','a','c','d']`. What happens when you add `a + b`?**

# DataFrame Basics
DataFrames are extensions of series into tables. They can have multiple indices (rows) and columns. Think of data frames as horizontally stacked series sharing the same set of indices

In [ ]:
#Generates default integer indexes (rows) and columns
df = pd.DataFrame(np.random.random((5,5)))
df

In [ ]:
#Can pass in the index (row) labels, as well as column labels instead
df_idx = pd.date_range('2015-01-01','2015-01-05',freq='d')
df_col = ['sun','mon','tues','wed','thurs']
df = pd.DataFrame(np.random.random((5,5)),index=df_idx, columns=df_col)
df

In [ ]:
intIndex = np.random.randint(0,100,10)
df2 = pd.DataFrame({'a': 1.,
                    'b': pd.Timestamp('2015-01-01'),
                    'c': pd.Series(np.random.random(10),index=intIndex),
                    'd': 'foo'},
                  index=intIndex)
df2

In [ ]:
#Multiple columnwise aggregations available. More available too! View docs
df.head()
df.tail()
df.mean()
df.min()
df.cumsum()
df.describe()

In [ ]:
df.columns #List the column labels of df

In [ ]:
df.index # List the rows labels of df

In [ ]:
#Unlike numpy, there is no row x column addressing using only [ ]
#i.e. x[2,4] would work for a numpy array. x[2,4] does NOT work for a pandas dataframe
df[0,0] #Error

In [ ]:
df.sun # the columns can be addressed directly as pandas series

In [ ]:
df['sun'] # this is also valid. This returns a pd Series

In [ ]:
df[ ['sun'] ] # Index by label vs. array returns different types.

In [ ]:
df[ ['sun','mon'] ] # Can just select certain columns

In [ ]:
#Since addressing single column returns a Series
# We can call the Series description fn's on them
df[ 'sun' ].value_counts()

In [ ]:
df['fri'] = 1.0* df['wed'] + 2.0 * df['thurs']
df

Be careful here. axis is switched with pandas. Here, axis=0 refers to rows, axis=1 refers to columns. Drop requires an `index label`

In [ ]:
df = df.drop('fri',axis=1)

In [ ]:
df = df.drop('2015-01-01',axis=0) #wont work. '2015-01-01' is not the right type

In [ ]:
df.index

In [ ]:
df = df.drop( pd.Timestamp('2015-01-01'), axis = 0)

## Exercise

**1) Create a dataframe where the indexes are every day from 1/1/2015 to 12/31/2015 and the columns are `calories` and `weight`. Calories should be uniform random integers from 1400 to 2000. Weight should be random normal generated having mean 180 and variance 20.**

Hint: Use np.random.randint and np.random.normal

**2) Add a new column `mood` to the dataframe having random uniform floats from 0 to 1.**

Hint: Use np.random.rand

**3) It turns out having data points everyday is really noisy. Resample the data to monthly estimates. Use method of mean to resample**

You can also create dataframes using a dictionary of objects for each column. Again, the index need not be string types

# Subsetting  Dataframes

Subsetting dataframes works similarly to numpy, but with some additional functionality

In [ ]:
df_idx = pd.date_range('2015-01-01','2015-01-05',freq='d')
df_col = ['sun','mon','tues','wed','thurs']
df = pd.DataFrame(np.random.random((5,5)),index=df_idx, columns=df_col)
df

In [ ]:
df[ df['sun'] > .5 ] #Subset certain rows, where the 'sun' column for that row is greater than .5

In [ ]:
df[ (df['sun'] > .5) & (df['mon'] < .5) ] #multiple conditions, use tuples for each condition

#### Indexing functions summary

Pandas Dataframes support various methods for indexing:

- .iloc <- Index by integer/positional
- .loc  <- Index by labels [can be integer labels!]

- .ix   <- Supports both label and integer/positional indexing. Tries to go by label first, and then positional indexing

In [ ]:
# .iloc grabs by Positional indexing
df.iloc[0] #Grab the 0th index row

In [ ]:
df.iloc[0:3] #Grab the 0 to 3th index row (NON INCLUSIVE END)

In [ ]:
# .iloc can grab subsets of the dataframe through slicing
df.iloc[0:3, 1:3]

In [ ]:
# .loc grabs by label
df.loc['2015-01-01']

In [ ]:
# .loc can also grab slices (INCLUSIVE END)
df.loc['2015-01-01':'2015-01-03']

In [ ]:
# .loc can grab subsets of the dataframe through slicing
df.loc['2015-01-01':'2015-01-03',['mon','tues']]

In [ ]:
# .ix allows for both label and positional indexing
df.ix['2015-01-01']
df.ix[0]

In [ ]:
# .ix can also mix label and positional indexing
df.ix[1:3,['sun','mon','thurs']]

What about adding rows into the dataframe using `iloc`, `loc`, and `ix`?

In [ ]:
#Iloc does not work. What would the index label have been for this anyway?
df.iloc[5] = pd.Series(np.random.rand(5),index=df.columns)

In [ ]:
#Loc works as of Pandas .13
df.loc[pd.Timestamp('2015-01-06')] = pd.Series(np.random.rand(5),index=df.columns)

In [ ]:
#ix also works when using label.
df.ix[pd.Timestamp('2015-01-07')] = pd.Series(np.random.rand(5),index=df.columns)

In [ ]:
# Again, even if using ix, if we pass in a positional label, wont work
df.ix[7] = pd.Series(np.random.rand(5),index=df.columns)

**The following two blocks are important when using integer indexes. Make sure you understand, or your code can be prone to bugs!**

* When you have a pure integer based index, `loc` will look for the integer label you specify. i.e. df.loc[3] will look for the row with index label == 3.
* When you have a pure integer based index, `iloc` will use the positional indexes you specify. i.e. df.iloc[3] will return the 4th row.
* When you have a pure integer based index, `ix` will look for the integer label you specify. i.e. df.ix[3] will look for the row with index label == 3
* When you have a mixed index containing integers, `ix` will use positional indexing! i.e. df.ix[3] will return the 4th row.


More information at:
[Good explanation for loc vs iloc vs ix](http://stackoverflow.com/questions/31593201/pandas-iloc-vs-ix-vs-loc-explanation/31593712#31593712)

In [ ]:
df = pd.Series(1, index=[49,48,47,46,45,1,2,3,4,5])
print df, '\n'
print 'Positional locate:'
print df.iloc[:3], '\n' #Positional locate.
print 'Label locate:'
print df.loc[:3], '\n' #Label locate. Goes up to and includes label
print 'Ix locate:'
print df.ix[:3], '\n'  #Tries first to do label loc.

print df.iloc[:6], '\n'
#print df.ix[:6]   #Fails. There is no label loc

#If labels, then use loc
#If integers, then use iloc
#Use ix when you have to mix integers and labels

#When there are mixed types in the index, ix falls back to positional indexing
df = pd.Series(1, index=['a','b','c',5,6,21])
print df, '\n'
print df.ix[:5] #Falls back to positional indexing!

In [ ]:
df = pd.DataFrame(np.random.random((3,2)),index=[100,200,300],columns=['A','B'])
print df, '\n'
print df.iloc[0], '\n'
print df.loc[100], '\n'
print df.ix[0] #This wont work because the index is integer based, and ix looks for label==0

## Exercise

**1) Create a dataframe where the indexes are every day from 1/1/2015 to 1/10/2015 and the columns are `a` and `b`. Generate the datapoints randomly from your favorite random function**

**2) Grab the rows of the dataframe where date is between '2015-01-06' and '2015-01-09' (inclusive) using `loc`**

**3) Grab the third through fifth rows of the dataframe (exclusive) using `iloc`**

**4) Insert a row of random numbers into the data frame having date 2015-01-11. **

**Hint1: If you have trouble, remember the type of the index you are inserting
Hint2: Remember that Pandas adds rows by matching up index/column names**

# Multiple indexing
Pandas allows you to have more than one set of indices or columns


In [ ]:
df3 = pd.DataFrame(np.random.randn(30,5),
                   index=pd.date_range('2015-1-1','2017-7-1',freq='m'))

In [ ]:
df3['blah'] = ['b1','b2','b3']*10
df3.head()

In [ ]:
df3 = df3.reset_index() #Reset the index of df3, set it as a new column
df3

In [ ]:
df3 = df3.set_index(['blah','index'])
df3

In [ ]:
df3.index.names = ['blah','date']
df3.head()

In [ ]:
df3.index #The Index type is a "MultiIndex" having a list of indexes

In [ ]:
df3.loc['b1'] #Works!

In [ ]:
df3.loc[ [pd.Timestamp('2015-01-31')]  ] #doesnt work

In [ ]:
df3.loc[('b1','2015-01-31')] # works

In [ ]:
# You can address up until the left most unaddressed level
df4 = pd.DataFrame(np.random.randn(8),index=['idx'+str(x) for x in range(8)])
df4['foobar'] = ['foo','foo','bar','bar']*2
df4['fahfoo'] = ['fah','bah']*4
df4

In [ ]:
df4 = df4.reset_index().set_index(['fahfoo','foobar','index'])
df4.index.names = ['fahfoo','foobar','idx3']
df4

In [ ]:
df4.loc[ ('fah','foo') ]

# Group By: Split-Apply-Combine
Pandas provides for powerful aggregation within 'groups'. The process involves:
* **Splitting** the data into groups based on criteria
* **Applying** a function to each of the groups independently
* **Combining** the groups back together into a dataframe

The **Apply** step can be any function such as Aggregating values (mean,min,median,count,etc), Transforming values (similar to the winsorization example), or Filtration (removing data)

The most similar paradigm would be SQL based statements such as:
```
SELECT column1, mean(column2), max(column3)
FROM TheTable
GROUP BY column1, column2
```

In [ ]:
df = pd.DataFrame({'A': ['foo','bar','foo','bar',
                         'foo','bar','foo','foo'],
                   'B': ['one','one','two','three',
                         'two','two','one','three'],
                   'C': np.random.randint(0,10,8),
                   'D': np.random.randint(0,10,8)})
df

`Groupby` objects are essentially mappings between 'groupings' and the set of indices the grouping is associated with. `Groupby` does NOT split. It just validates a correct mapping of labels to group names

In [ ]:
grouped = df.groupby('A') #groupby object. Group by the unique A's
for name, group in grouped:
    print "Name:", name
    print "Group:"
    print group, "\n"

In [ ]:
grouped = df.groupby(['A','B']) # Group by unique combn of A's and B's
for name, group in grouped:
    print "Name:", name
    print "Group:"
    print group, "\n"

In [ ]:
#Can also split on rows. You can specify your own rules
grouped = df.groupby(lambda x: 'Even' if x%2==0 else 'Odd', axis=0)

for name, group in grouped:
    print "Name:", name
    print "Group:"
    print group, "\n"

In [ ]:
grouped.groups #Get mapping of group to the LABELS in each group

In [ ]:
#Can also split on columns. You can specify your own rules
def tmp(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'

grouped = df.groupby(tmp, axis=1)
grouped.get_group('consonant')

In [ ]:
grouped.groups #Get mapping of group to the LABELS in each group

In [ ]:
#Another example, doing grouping on the columns
multiIdx = pd.MultiIndex.from_tuples(    
    list(zip(*[ ['fah','bah']*4,['foo','foo','bar','bar']*2])),
    names=['fahbah','foobar'])

df4 = pd.DataFrame({'A': np.random.randint(0,10,8),
                    'B': np.random.randint(0,10,8)},
                   index=multiIdx)

grouped = df4.groupby(level=0)
df4

In [ ]:
#Various descriptive stats measured on each of the groups
grouped.all() # All of the elements are true (or coercible to true)
grouped.any()
grouped.count()
grouped.count()
grouped.mean()

In [ ]:
# You can specifiy an aggregation functions
grouped.agg(np.mean) #Specify existing functions

#Or even write your own
grouped.agg( lambda x: np.sum(x)/np.float64(len(x)))

In [ ]:
# can handle multiple aggregations on the columns through dicts
grouped.agg({'A': ['mean','min','max'], 'B': ['count','nunique']})

In [ ]:
#you can also do in place transformations of the data
def zScore(x):
    return (x - x.mean()) / x.std()

grouped = df4.groupby(level=1)
#Group by 'fah' and 'bah', take all of the elements in them and zscore
grouped.transform(zScore) 

If using transform, check that your transformation function actually returns the same number of elements

In [ ]:
#Does not do what you think. 
#Notice that each element got of 'fah' got the same mean
grouped.transform(np.mean)

In [ ]:
#apply is similar to transform, but resulting shape not necessarily same
df4.groupby(level=0).apply(lambda x: (x.max(), x.min()) )

# What?

`apply` applies a function to each group. The sizes can be different.

`agg` applies a function to each column for each group to reduce down to a single value.

`transform` applies a function to each group, but the result must be the same size as that which is passed in. (ideal for: standardization, winsorization, demeaning, filling NAs with the mean etc)

For more information: [Groupby](http://pandas.pydata.org/pandas-docs/stable/groupby.html)

## Exercise

Pandas also has the ability to dynamically download datasets using the read_csv function.

Download the following dataset using the following command. Note that this may take a while--you'll know if Python is still running if there is an asterisk to the left of the command
```
import pandas as pd
chi = pd.read_csv('https://data.cityofchicago.org/api/views/4ijn-s7e5/rows.csv?accessType=DOWNLOAD')
chi.head()
```

What is the shape of this data set? How many rows and columns are there?

Hint: .info() or shape

How many distinct cities are in this dataset?

What is the most common Inspection Type? Hint: Use `groupby` and `idxmax`